In [5]:
import json
import pandas as pd
import csv
import re
import os
import pandasql as sql
from jira import JIRA, JIRAError
from datetime import datetime

## Read credentials from JSON file

In [3]:
# Reading credentials file JSON
with open('credential_jira.json') as f:
    credentials = json.load(f)

user = credentials['user']
password = credentials['apikey']
link = credentials['link']
project_key = credentials['project_key'] 


## Execute JQL query to obtain info from Jira Project

In [4]:
# Jira authentication 
jira = JIRA(server=link, basic_auth=(user, password))

# Querying data using JQL language
jql_query = f'project = {project_key} ORDER BY created ASC'

# Pagination of all isseus from Jira project
block_size = 100
block_num = 0

# Opening CSV file for writing all issues in there
print(f"Extracting tickets from {project_key} Jira project...")

with open('jira_issues.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Writing CSV Column names
    writer.writerow([
        'key', 
        'tool_value',
        'area',
        'sprint',
        'summary', 
        'status', 
        'assignee', 
        'reporter', 
        'created', 
        'updated', 
        #'custom_create_date',
        #'custom_end_date',
        'date_diff_days',
        'priority', 
        'issue_type', 
        #'labels', 
        #'comments', 
    ])

    while True:
        start_idx = block_num * block_size
        try:
            issues = jira.search_issues(jql_query, startAt=start_idx, maxResults=block_size, expand='changelog')
        except JIRAError as e:
            if e.status_code == 404:
                print(f"Ignoring not found tickets {block_num}")
                break  # Quit while if face 404 error
            else:
                raise  # Rerun exception if error was |= from 404 error
        else:
            if not issues:
                break
            for issue in issues:
                # RegExing main custom fileds
                tool_pattern = r"value='(.*?)'"
                tool_match = re.search(tool_pattern, str(issue.fields.customfield_11897)) #Tool
                tool_value = tool_match.group(1) if tool_match else None

                area_pattern = r"value='(.*?)'"
                area_match = re.search(area_pattern, str(issue.fields.customfield_11896)) #Area
                area_value = area_match.group(1) if area_match else None

                sprint_field = issue.fields.customfield_10007
                if isinstance(sprint_field, list):
                    sprint_value = ', '.join([str(sprint) for sprint in sprint_field])
                else:
                    sprint_value = str(sprint_field)

                # Converting dates (Created e Updated) in a date filed
                created_date = datetime.strptime(issue.fields.created, "%Y-%m-%dT%H:%M:%S.%f%z")
                updated_date = datetime.strptime(issue.fields.updated, "%Y-%m-%dT%H:%M:%S.%f%z")
                date_diff_days = (updated_date - created_date).days

                #custom_create_date = datetime.strptime(issue.fields.customfield_11930, "%Y-%m-%dT%H:%M:%S.%f%z")
                #custom_end_date = datetime.strptime(issue.fields.customfield_11931, "%Y-%m-%dT%H:%M:%S.%f%z")

                writer.writerow([
                    issue.key,
                    tool_value,
                    area_value,
                    sprint_value,
                    issue.fields.summary,
                    issue.fields.status.name,
                    issue.fields.assignee.displayName if issue.fields.assignee else 'Unassigned',
                    issue.fields.reporter.displayName,
                    created_date,
                    updated_date,
                    #issue.fields.customfield_11930, #custom_create_date
                    #issue.fields.customfield_11931, #custom_end_date
                    date_diff_days,
                    issue.fields.priority.name if issue.fields.priority else 'None',
                    issue.fields.issuetype.name,
                    #issue.fields.labels if issue.fields.labels else '',
                    #'\n'.join([comment.body for comment in issue.fields.comment.comments]) if issue.fields.comment.comments else '',
                ])
            block_num += 1

print(f"All the tickets where created and written in jira_issues.csv")


Extracting tickets from IBRR Jira project...
All the tickets where created and written in jira_issues.csv
